# 循环网络模型

In [27]:
import csv
import re
from abc import abstractmethod, ABC

import numpy as np

np.random.seed(99)

## Foundation

### Tensor

In [28]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        topo = []
        visited = set()

        def build_topo(t):
            if t not in visited:
                visited.add(t)
                for p in t.parents:
                    build_topo(p)
                topo.append(t)

        build_topo(self)

        self.grad = np.ones_like(self.data)
        for t in reversed(topo):
            t.gradient_fn()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return str(self.data)

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)
            self.grad += grad[0]
            other.grad += grad[1]

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

### Base Dataset

In [29]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size
        return Tensor(self.features[start: end]), Tensor(self.labels[start: end])

    @abstractmethod
    def estimate(self, predictions):
        pass

### Base Layer

In [30]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [31]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [32]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### Base Model

In [33]:
class Model(ABC):

    def __init__(self, layer, loss, optimizer):
        self.layer = layer
        self.loss = loss
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## Data

### IMDB Dataset

In [34]:
class IMDBDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = []
        self.labels = []
        for line in self.tokens[:-10]:
            for index in range(len(line) - 4):
                self.features.append([line[index], line[index + 1], line[index + 3], line[index + 4]])
                self.labels.append(self.onehot(line[index + 2]))

    def eval(self):
        self.features = []
        self.labels = []
        for line in self.tokens[-10:]:
            for index in range(len(line) - 4):
                self.features.append([line[index], line[index + 1], line[index + 3], line[index + 4]])
                self.labels.append(self.onehot(line[index + 2]))

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def onehot(self, token):
        ebd = np.zeros(len(self.vocabulary))
        ebd[token] = 1
        return ebd

    @staticmethod
    def argmax(vector):
        return [np.argmax(vector)]

    def estimate(self, predictions):
        count = 0
        for i in range(len(predictions)):
            if self.argmax(predictions[i].data[0]) == self.argmax(self.labels[i].data):
                count += 1
        return count / len(predictions)

## Model

### Linear Layer

In [35]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data / len(x.data)
            self.bias.grad += np.sum(p.grad, axis=0) / len(x.data)
            x.grad += p.grad @ self.weight.data / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### Sequential Layer

In [36]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### Embedding Layer

In [37]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.randn(embedding_size, vocabulary_size) * np.sqrt(2 / vocabulary_size))

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if type(self.weight.grad) is not np.ndarray:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

### ReLU Activation Function

In [38]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += (p.data > 0) * p.grad

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Tanh Activation Function

In [39]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad += p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Sigmoid Activation Function

In [40]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Softmax Activation Function

In [41]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Recurrent Layer

In [42]:
class Recurrent(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.embedding = Embedding(vocabulary_size, embedding_size)
        self.update = Linear(embedding_size * 2, embedding_size)
        self.hidden = Linear(embedding_size, embedding_size)
        self.output = Linear(embedding_size, vocabulary_size)
        self.tanh = Tanh()

        self.layers = [self.embedding,
                       self.update,
                       self.hidden,
                       self.output,
                       self.tanh]

    def forward(self, x: Tensor, h: Tensor):
        if not h:
            h = Tensor(np.zeros((1, self.embedding_size)))

        embedding_feature = self.embedding(x)
        concat_feature = self.tanh(embedding_feature.concat(h, axis=1))
        cell_hidden = self.tanh(self.update(concat_feature))
        hidden_feature = self.tanh(self.hidden(cell_hidden))

        return (self.output(hidden_feature),
                Tensor(hidden_feature.data))

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

### MSE Loss Function

In [43]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### Cross Entropy Loss Function

In [44]:
class CELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(p.data))

        def gradient_fn():
            p.grad += (softmax - y.data) / len(p.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### Binary Cross Entropy Loss Function

In [45]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped)
                              + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped) * len(p.data))

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### SGD Optimizer

In [46]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### Recurrent Neural Network Model

In [47]:
class RNNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            hidden = None
            for i in range(len(dataset)):
                feature, label = dataset[i]

                prediction, hidden = self.layer(feature, hidden)
                error = self.loss(prediction, label)

                self.optimizer.reset()
                error.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        predictions = []
        hidden = None
        for i in range(len(dataset)):
            feature, label = dataset[i]
            prediction, hidden = self.layer(feature, hidden)
            predictions.append(prediction)
        return predictions

## Configuration

### Learning Rate

In [48]:
LEARNING_RATE = 0.01

### Epoch

In [49]:
EPOCHS = 100

## Training

### Iteration Training

In [50]:
dataset = IMDBDataset('tinyimdb.csv')
layer = Recurrent(len(dataset.vocabulary), 64)
loss = CELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = RNNModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)

## Testing

### Estimating

In [51]:
predictions = model.test(dataset)

print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 0.6687898089171974


### Comparing Labels and Predictions

In [52]:
features, labels = dataset.items()
for i in range(len(predictions)):
    pos = np.argmax(predictions[i].data[0])
    print(f'Feature: {dataset.decode(features.data[i])} | '
          f'Label: {dataset.decode(dataset.argmax(labels.data[i]))} | '
          f'Prediction: {dataset.decode(dataset.argmax(predictions[i].data[0]))}')

Feature: worst movie awful music | Label: with | Prediction: the
Feature: movie with music the | Label: awful | Prediction: boring
Feature: with awful the actor | Label: music | Prediction: music
Feature: awful music actor did | Label: the | Prediction: the
Feature: music the did a | Label: actor | Prediction: director
Feature: the actor a boring | Label: did | Prediction: did
Feature: actor did boring job | Label: a | Prediction: a
Feature: did a job actress | Label: boring | Prediction: fantastic
Feature: a boring actress director | Label: job | Prediction: job
Feature: boring job director character | Label: actress | Prediction: actress
Feature: job actress character screenplay | Label: director | Prediction: director
Feature: actress director screenplay scene | Label: character | Prediction: character
Feature: director character scene but | Label: screenplay | Prediction: screenplay
Feature: character screenplay but or | Label: scene | Prediction: scene
Feature: screenplay scene or